## SQL Database

### 🗄️ SQLite Database Processing — LangChain + Custom SQL

🎯 Purpose
- Work with SQLite database tables
- Convert database rows into documents
- Support both utility-based loading and custom full-control processing

✅ Method 1 — SQLDatabase Utility (Simple & Quick)
Use when:
- You want to inspect database tables quickly
- Need schema + table info for exploration
- Suitable for testing / demos / debugging

What happens:
- Connects to SQLite using SQLDatabase.from_uri()
- Retrieves:
- - table names

- - table schema / structure

- Helps understand database before processing

🧠 Think of it as: “Quick database overview, minimal control”

✅ Method 2 — Custom SQL Processing (Full Control)
Use when:
- You want custom SQL queries
- Need JOIN logic across tables
- Want readable page_content formatting
- Need rich metadata fields
- Suitable for RAG pipelines / production

What happens:
- Execute your own SQL query
- Loop through rows
- Build human-readable content
- Attach structured metadata
- Convert rows → Document objects

🧠 Think of it as: “SQL records become structured knowledge documents”

🧾 Example Use-Cases
- Convert database tables to embeddings-ready text
- Group records by department / category
- Enrich records using joins
- Export DB content as JSON / JSONL / chunks

🚀 Key Takeaways
- SQLDatabase Utility = simple inspection & exploration
- Custom SQL Processing = maximum control + flexibility
- Best choice depends on speed vs structure needs

In [1]:
import sqlite3

# Connect to database
conn = sqlite3.connect("data/databases/sample.db")
cursor = conn.cursor()


In [2]:
# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    role TEXT,
    experience_years INTEGER,
    location TEXT
);
""")

In [3]:
# Create second table
cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT NOT NULL,
    manager TEXT
);
""")

In [4]:
# Sample data to insert
employees_data = [
    (1, "Aisha", "Data Analyst", 2, "Bangalore"),
    (2, "Rahul", "ML Engineer", 3, "Hyderabad"),
    (3, "Meera", "Product Manager", 4, "Remote"),
    (4, "Karan", "Backend Developer", 1, "Pune"),
    (5, "Sara", "HR Executive", 2, "Delhi")
]

departments_data = [
    (1, "Engineering", "Rahul Mehta"),
    (2, "Data", "Aisha Khan"),
    (3, "HR", "Meera Kapoor")
]

In [5]:
cursor.executemany(
    "INSERT INTO employees (id, name, role, experience_years, location) VALUES (?, ?, ?, ?, ?)",
    employees_data
)
cursor.executemany(
    "INSERT INTO departments (dept_id, dept_name, manager) VALUES (?, ?, ?)",
    departments_data
)

In [6]:
cursor.execute("SELECT * FROM employees")

In [6]:
conn.commit()
conn.close()

## Database Content Extraction

In [7]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

c:\Users\Ahmed\OneDrive\Desktop\ExploringRAGs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
## Method:1 SQLDatabas utility
db = SQLDatabase.from_uri("sqlite:///data/databases/sample.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable Info")
print(db.get_table_info())


Tables: ['departments', 'employees']

Table Info

CREATE TABLE departments (
	dept_id INTEGER, 
	dept_name TEXT NOT NULL, 
	manager TEXT, 
	PRIMARY KEY (dept_id)
)

/*
3 rows from departments table:
dept_id	dept_name	manager
1	Engineering	Rahul Mehta
2	Data	Aisha Khan
3	HR	Meera Kapoor
*/


CREATE TABLE employees (
	id INTEGER, 
	name TEXT NOT NULL, 
	role TEXT, 
	experience_years INTEGER, 
	location TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	experience_years	location
1	Aisha	Data Analyst	2	Bangalore
2	Rahul	ML Engineer	3	Hyderabad
3	Meera	Product Manager	4	Remote
*/


In [10]:
# ✅ Custom SQL Processing — Full Control (Recommended)
import sqlite3
from langchain_core.documents import Document

conn = sqlite3.connect("data/databases/sample.db")
cursor = conn.cursor()

# Query data (you control what to fetch)
cursor.execute("""
SELECT e.id, e.name, e.role, e.experience_years, e.location,
       d.dept_name, d.manager
FROM employees e
LEFT JOIN departments d
ON e.id % 3 + 1 = d.dept_id   -- sample mapping for demo
""")

rows = cursor.fetchall()
conn.close()

documents = []

for row in rows:
    emp_id, name, role, exp, location, dept_name, manager = row

    # Page content (human-readable text)
    content = f"""
    Name: {name}
    Role: {role}
    Experience: {exp} years
    Location: {location}
    Department: {dept_name}
    Manager: {manager}
    """

    # Structured metadata
    metadata = {
        "id": emp_id,
        "department": dept_name,
        "manager": manager,
        "source": "sample.db",
        "table": "employees"
    }

    documents.append(
        Document(
            page_content=content.strip(),
            metadata=metadata
        )
    )

print(f"Created {len(documents)} SQL-derived documents")
print(documents[0])
print(documents)


Created 5 SQL-derived documents
page_content='Name: Aisha
    Role: Data Analyst
    Experience: 2 years
    Location: Bangalore
    Department: Data
    Manager: Aisha Khan' metadata={'id': 1, 'department': 'Data', 'manager': 'Aisha Khan', 'source': 'sample.db', 'table': 'employees'}
[Document(metadata={'id': 1, 'department': 'Data', 'manager': 'Aisha Khan', 'source': 'sample.db', 'table': 'employees'}, page_content='Name: Aisha\n    Role: Data Analyst\n    Experience: 2 years\n    Location: Bangalore\n    Department: Data\n    Manager: Aisha Khan'), Document(metadata={'id': 2, 'department': 'HR', 'manager': 'Meera Kapoor', 'source': 'sample.db', 'table': 'employees'}, page_content='Name: Rahul\n    Role: ML Engineer\n    Experience: 3 years\n    Location: Hyderabad\n    Department: HR\n    Manager: Meera Kapoor'), Document(metadata={'id': 3, 'department': 'Engineering', 'manager': 'Rahul Mehta', 'source': 'sample.db', 'table': 'employees'}, page_content='Name: Meera\n    Role: Produc